multi output LSTM (network)

In [ ]:
import pandas as pd
import numpy as np
from numpy import array
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# parameters
num_features = 20
num_scenarios = 6
scenario_length = 1000
attack_type = 0 		# 0: all, 1: DoS, 2: FDI, 3: Replay
num_attack_types = 3 + 1
num_attack_targets = 6 + 1
num_quad = 2

# hyper-parameter
seq_len = 20
seq_overlap = seq_len - 1
lstm_blocks = 64
epoch_val = 20
batch_size_val = 4

# 1 data preprocessing
data = pd.read_csv('dataset.csv')

# 1.1 normalization
labels = data[['label_1', 'type_1', 'target_1', 'label_2', 'type_2', 'target_2']]
data = data.drop(columns=['time', 'label_1', 'type_1', 'target_1', 'label_2', 'type_2', 'target_2'])
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data)
data_normalized = pd.DataFrame(data_normalized, columns=data.columns)
data_normalized = pd.concat([data_normalized, labels], axis=1)

# 1.2 sequence generation
sequences = []
step_len = seq_len - seq_overlap

if attack_type == 0:
    scenario_range = range(0, 6)
elif attack_type == 1:
    scenario_range = range(0, 2)
elif attack_type == 2:
    scenario_range = range(2, 4)
elif attack_type == 3:
    scenario_range = range(4, 6)

for s in scenario_range:
    scenario_start = s * scenario_length
    for i in range(scenario_start, scenario_start + scenario_length - seq_len, step_len):
        sequence = data_normalized[i:i + seq_len]
        sequences.append(sequence)
data_sequences = array(sequences)

# 2 data preperation
# 2.1 train-test split
data_reshaped = data_sequences.reshape(data_sequences.shape[0], -1)
X_train, X_test = train_test_split(data_reshaped, test_size=0.25, random_state=42)
X_train = X_train.reshape(X_train.shape[0], data_sequences.shape[1], data_sequences.shape[2])
X_test = X_test.reshape(X_test.shape[0], data_sequences.shape[1], data_sequences.shape[2])

# 2.2 reshape data for LSTM network
y_train_detection_1 = X_train[:, -1, -6]
y_train_identification_1 = X_train[:, -1, -5]
y_train_isolation_1 = X_train[:, -1, -4]
y_train_detection_2 = X_train[:, -1, -3]
y_train_identification_2 = X_train[:, -1, -2]
y_train_isolation_2 = X_train[:, -1, -1]
X_train = X_train[:, :, :-6]

y_test_detection_1 = X_test[:, -1, -6]
y_test_identification_1 = X_test[:, -1, -5]
y_test_isolation_1 = X_test[:, -1, -4]
y_test_detection_2 = X_test[:, -1, -3]
y_test_identification_2 = X_test[:, -1, -2]
y_test_isolation_2 = X_test[:, -1, -1]
X_test = X_test[:, :, :-6]

# 3. model creation
# 3.1 model archiecture
input_layer = Input(shape=(seq_len, num_features))
shared_lstm = LSTM(lstm_blocks)(input_layer)
output_detection_1 = Dense(1, activation='sigmoid', name='detection_output_1')(shared_lstm)
output_identification_1 = Dense(num_attack_types, activation='softmax', name='identification_output_1')(shared_lstm)
output_isolation_1 = Dense(num_attack_targets, activation='softmax', name='isolation_output_1')(shared_lstm)
output_detection_2 = Dense(1, activation='sigmoid', name='detection_output_2')(shared_lstm)
output_identification_2 = Dense(num_attack_types, activation='softmax', name='identification_output_2')(shared_lstm)
output_isolation_2 = Dense(num_attack_targets, activation='softmax', name='isolation_output_2')(shared_lstm)

model = Model(inputs=input_layer, outputs=[output_detection_1, output_identification_1, output_isolation_1,
                                           output_detection_2, output_identification_2, output_isolation_2])

# 3.2 model compile
model.compile(
    loss={
        'detection_output_1': 'binary_crossentropy',
        'identification_output_1': 'sparse_categorical_crossentropy',
        'isolation_output_1': 'sparse_categorical_crossentropy',
        'detection_output_2': 'binary_crossentropy',
        'identification_output_2': 'sparse_categorical_crossentropy',
        'isolation_output_2': 'sparse_categorical_crossentropy'
    },
    optimizer='adam',
    metrics={
        'detection_output_1': ['accuracy'],
        'identification_output_1': ['accuracy'],
        'isolation_output_1': ['accuracy'],
        'detection_output_2': ['accuracy'],
        'identification_output_2': ['accuracy'],
        'isolation_output_2': ['accuracy']
    }
)
print(model.summary())

# 3.3 model train
model.fit(X_train, [y_train_detection_1, y_train_identification_1, y_train_isolation_1,
                    y_train_detection_2, y_train_identification_2, y_train_isolation_2],
          epochs=epoch_val, batch_size=batch_size_val)

# 3.4 model test
y_pred_detection_1, y_pred_identification_1, y_pred_isolation_1, \
y_pred_detection_2, y_pred_identification_2, y_pred_isolation_2 = model.predict(X_test)

# Convert predictions for binary and multi-class
y_pred_detection_1 = (y_pred_detection_1 > 0.5).astype(int).reshape(-1)
y_pred_identification_1 = np.argmax(y_pred_identification_1, axis=1)
y_pred_isolation_1 = np.argmax(y_pred_isolation_1, axis=1)
y_pred_detection_2 = (y_pred_detection_2 > 0.5).astype(int).reshape(-1)
y_pred_identification_2 = np.argmax(y_pred_identification_2, axis=1)
y_pred_isolation_2 = np.argmax(y_pred_isolation_2, axis=1)

# 3.5 print results for each column

# quad 1
accuracy = accuracy_score(y_test_detection_1, y_pred_detection_1)
precision = precision_score(y_test_detection_1, y_pred_detection_1)
recall = recall_score(y_test_detection_1, y_pred_detection_1)
f1 = f1_score(y_test_detection_1, y_pred_detection_1)

print('Quad 1:')
print(' Detection:')
print(f'    Accuracy: {accuracy:.3f}')
print(f'    Precision: {precision:.3f}')
print(f'    Recall: {recall:.3f}')
print(f'    F1-score: {f1:.3f}')

# 3.6 print results for identification
accuracy_id = accuracy_score(y_test_identification_1, y_pred_identification_1)
print(' Identification:')
print(f'    Accuracy: {accuracy_id:.3f}')

# 3.7 print results for localization
accuracy_iso = accuracy_score(y_test_isolation_1, y_pred_isolation_1)
print(' Localization:')
print(f'    Accuracy: {accuracy_iso:.3f}')


# quad 2
accuracy = accuracy_score(y_test_detection_2, y_pred_detection_2)
precision = precision_score(y_test_detection_2, y_pred_detection_2)
recall = recall_score(y_test_detection_2, y_pred_detection_2)
f1 = f1_score(y_test_detection_2, y_pred_detection_2)

print('Quad 2:')
print(' Detection:')
print(f'    Accuracy: {accuracy:.3f}')
print(f'    Precision: {precision:.3f}')
print(f'    Recall: {recall:.3f}')
print(f'    F1-score: {f1:.3f}')

# 3.6 print results for identification
accuracy_id = accuracy_score(y_test_identification_2, y_pred_identification_2)
print(' Identification:')
print(f'    Accuracy: {accuracy_id:.3f}')

# 3.7 print results for localization
accuracy_iso = accuracy_score(y_test_isolation_2, y_pred_isolation_2)
print(' Localization:')
print(f'    Accuracy: {accuracy_iso:.3f}')


